In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from dotenv import load_dotenv
import os
import textwrap
import openai
load_dotenv()

/home/tguyot/PersonalCode/ragosophy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# Already chunked Marcus Aurelius corpus
df = pd.read_csv('cleaned/marcus_aurelius_paragraphs.csv')

In [3]:
df

,livre,paragraph,text
0,1,1,Exemples que j’ai reçus de mon grand-père Véru...
1,1,2,Du père qui m’a donné la vie : la modestie et ...
2,1,3,De ma mère : la piété et la générosité ; l’hab...
3,1,4,"À mon bisaïeul, je suis redevable de n’avoir p..."
4,1,5,"À mon gouverneur, de n’avoir jamais été de la ..."
...,...,...,...
482,12,32,Quelle infime parcelle chacun des êtres n’ont-...
483,12,33,Quel usage ton âme fait-elle d’elle-même ? Tou...
484,12,34,Rien ne peut nous inspirer plus sûrement le mé...
485,12,35,Quand on ne trouve bon que ce qui vient en son...


In [4]:
# Embedding model
model = SentenceTransformer("dangvantuan/sentence-camembert-base")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 1995.61it/s, Materializing param=pooler.dense.weight]                               
CamembertModel LOAD REPORT from: dangvantuan/sentence-camembert-base
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [4]:
chunks = df['text'].tolist()

In [6]:
# Embed
embeddings = model.encode(chunks, convert_to_tensor=True, )

In [7]:
embeddings.shape

torch.Size([487, 768])

In [8]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings())
collection = client.create_collection(name="marcus_aurelius")

In [9]:
collection.add(
    ids=[str(i) for i in range(len(chunks))],
    documents=chunks,
    embeddings=embeddings.cpu().numpy().tolist(
)
)

In [10]:
question = "l'exemple d'Alexandre le grammairien"
embed_question = model.encode(question, convert_to_tensor=True)

In [11]:
result = collection.query(
    query_embeddings=embed_question.cpu().numpy().tolist(),
    n_results=5)
result

{'ids': [['9', '400', '82', '230', '11']],
 'embeddings': None,
 'documents': [['Par l’exemple d’Alexandre le grammairien, j’ai appris à ne jamais choquer les gens, à ne les point heurter par une brusquerie blessante pour un barbarisme qu’ils auraient commis, pour une tournure fautive ou une prononciation vicieuse qui leur serait échappée  ; mais à m’arranger adroitement dans la conversation pour que le mot qui aurait dû être choisi d’abord reparût, par manière de réponse ou de confirmation, en donnant mon avis sur la chose même sans m’arrêter du tout à l’expression malheureuse, ou en prenant soigneusement tel autre détour pour dissimuler l’allusion.',
   'Répète-toi sans cesse que la manière dont les choses se passent actuellement est aussi la manière dont elles se passaient jadis, dont elles se passeront plus tard. Remets-toi sous les yeux tous ces drames, tous ces théâtres, toujours si uniformes, que tu as pu connaître, soit par ton expérience personnelle, soit par les récits d’une 

In [12]:
def retrieve_context(question, collection, top_k=5):
    embed_question = model.encode(question, convert_to_tensor=True)
    result = collection.query(
        query_embeddings=embed_question.cpu().numpy().tolist(),
        n_results=top_k)
    
    context = "\n".join(result['documents'][0])
    return context

In [13]:
retrieve_context('Alexandre le grammairien et son exemple Platon Platon ', collection)

'Extrait de Platon : «  Mais crois-tu que celui dont la pensée est pleine de grandeur, et qui contemple tous les temps et tous les êtres, puisse regarder la vie qu’on passe ici-bas comme quelque chose de bien important ? « — C’est impossible. « — Ainsi la mort ne devra pas lui paraître à craindre ? « — Non. »\nLa cause universelle est un torrent qui entraîne toutes choses. Aussi, qu’ils sont naïfs même ces prétendus hommes d’État qui s’imaginent régler par la philosophie la pratique des affaires ! Ce sont des enfants qui ont encore la morve au nez. Ô homme, que te faut-il donc ? Borne-toi à faire ce que présentement la nature exige. Agis, puisque tu le peux  ; et ne t’inquiète pas de savoir si quelqu’un regarde ce que tu fais. Ne va pas espérer non plus la République de Platon ; mais sache te contenter du plus léger progrès ; et si tu réussis, ne crois pas avoir gagné si peu de chose. Qui peut en effet changer l’esprit des hommes  ? Et tant qu’on ne parvient pas à modifier les cœurs et

In [ ]:
# Now let's query a LLM : Openai

count_response = 0

openai.api_key = os.getenv("OPENAI_API_KEY")
wrapper = textwrap.TextWrapper(width=200)

question = input("Je suis Marc Aurèle. Posez-moi une question sur mes Méditations:\n")
while question.lower() != "merci" or count_response < 5:
  print('Votre question est :', question)

  response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=500,
    messages=[
      {"role": "system", "content": "Tu es un clone de Marc Aurèle qui répond aux questions sur les Méditations de Marc Aurèle, "
      "basé sur le contexte fourni et la propre façon de parler, d'écrire et de penser de Marc Aurèle. Réponds brièvement et avec sagesse, sans t'égarer dans "
      "tes propos. Si tu ne connais pas la réponse, dis-le franchement. Utilise un langage simple et direct. Inspire-toi des citations célèbres des Méditations."
      "Ne parle pas de toi en tant que modèle de langage. Sois humble et stoïque, et reste fidèle à la philosophie stoïcienne. Enfin, adopte un langage et un ton qui reflètent le style d'écriture de Marc Aurèle,"
      "tout en restant simple, accessible et compréhensible pour un public moderne."
      "Si le contexte fourni ne contient pas d'informations pertinentes, réponds que tu n'as pas la réponse. Si la question est hors sujet, dans une autre langue que le français, ou inappropriée, décline poliment de répondre."},
      {"role": "user", "content": f"Voici un contexte des Méditations de Marc Aurèle:\n{retrieve_context(question, collection)}"},
      {"role": "user", "content": question}
    ]
  )

  print(wrapper.fill(response.choices[0].message.content))

  print('Saisir "Merci" pour terminer la discussion')

  question = input('Une autre question ?\n')

print("Je vous en prie. Adieu!")


Votre question est : De quoi peux-tu me parler ?
Je peux te parler de la raison, de la maîtrise de soi, de la persévérance face aux épreuves, de l'importance de se conformer à sa nature propre, de l'indifférence aux choses extérieures qui ne
dépendent pas de nous, de la sagesse de toujours chercher à faire et dire ce qu'il y a de mieux dans chaque situation, et de la capacité de la raison à surmonter tous les obstacles.
Saisir "Merci" pour terminer la discussion
Votre question est : C'ets quoi la raison ?
La raison, c'est la faculté de discernement et de compréhension qui nous permet de distinguer le bien du mal, le juste de l'injuste. C'est la capacité à réfléchir de manière logique et rationnelle, à
contrôler nos émotions et à agir de manière réfléchie et vertueuse. La raison nous guide vers la sagesse, la modération et la conformité à la nature. Elle nous aide à nous élever au-dessus de nos
passions et de nos désirs pour atteindre un état de tranquillité et de sérénité intérieure. C

KeyboardInterrupt: Interrupted by user